In [1]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


## 충남 Cluster 3

In [4]:
chungnam_3 = pd.read_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/modeling data/chungnam_3.csv')
display(chungnam_3)
chungnam_3.isnull().sum()

,date,accident_num,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 이슬점온도(°C),year,month,day,측정소명,O3,PM10,PM2.5,season,요일,workday
0,2020-01-01,2.0,0.3,0.0,0.7,68.1,-5.0,2020,1,1,금산읍,0,1,1,winter,수요일,1
1,2020-01-02,4.0,1.8,0.0,0.8,73.4,-2.5,2020,1,2,금산읍,0,1,2,winter,목요일,0
2,2020-01-03,2.0,2.5,0.0,0.6,73.8,-1.9,2020,1,3,금산읍,0,2,2,winter,금요일,0
3,2020-01-04,2.0,1.8,0.0,0.9,76.0,-2.1,2020,1,4,금산읍,0,2,2,winter,토요일,1
4,2020-01-05,0.0,2.0,0.0,1.3,54.9,-6.8,2020,1,5,금산읍,0,1,2,winter,일요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2023-01-27,0.0,-6.4,0.0,2.1,65.1,-12.4,2023,1,27,금산읍,0,0,1,winter,금요일,0
1123,2023-01-28,0.0,-7.1,0.0,1.0,72.3,-11.7,2023,1,28,금산읍,0,0,1,winter,토요일,1
1124,2023-01-29,0.0,-1.2,0.2,1.6,68.0,-6.9,2023,1,29,금산읍,0,1,1,winter,일요일,1
1125,2023-01-30,0.0,-2.1,0.0,1.2,54.5,-11.0,2023,1,30,금산읍,0,1,1,winter,월요일,0


date            0
accident_num    0
평균기온(°C)        0
일강수량(mm)        0
평균 풍속(m/s)      0
평균 상대습도(%)      0
평균 이슬점온도(°C)    0
year            0
month           0
day             0
측정소명            0
O3              0
PM10            0
PM2.5           0
season          0
요일              0
workday         0
dtype: int64

In [5]:
chungnam_3['date'] = pd.to_datetime(chungnam_3['date'])
chungnam_3['accident_hpn'] = chungnam_3['accident_num'].apply(lambda x: 1 if x > 0 else 0)
chungnam_3['accident_hpn'].value_counts()/chungnam_3['accident_hpn'].value_counts().sum()

1    0.704525
0    0.295475
Name: accident_hpn, dtype: float64

In [6]:
X = chungnam_3.drop(['date','accident_num','측정소명','accident_hpn'], axis = 1)
y = chungnam_3['accident_hpn']

In [7]:
print(X.shape)
print(y.shape)

(1127, 14)
(1127,)


In [8]:
X_trn, X_tes, y_trn, y_tes = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

print(X_trn.shape)
print(X_tes.shape)
print(y_trn.shape)
print(y_tes.shape)

(901, 14)
(226, 14)
(901,)
(226,)


In [9]:
smote = SMOTENC(random_state=0, categorical_features = list(range(5,14)))
X_res, y_res = smote.fit_resample(X_trn,y_trn)

In [10]:
print(y_res.value_counts())
print(y_res.value_counts()/y_res.value_counts().sum())

1    635
0    635
Name: accident_hpn, dtype: int64
1    0.5
0    0.5
Name: accident_hpn, dtype: float64


In [ ]:
cat_features = ['year', 'month','day','season','요일']
num_features = ['평균기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','평균 이슬점온도(°C)','O3','PM10','PM2.5']

enc = OneHotEncoder(handle_unknown = 'ignore')
scaler = MinMaxScaler()

enc.fit(X_res[cat_features])
scaler.fit(X_res[num_features])

train_oh = enc.transform(X_res[cat_features]).toarray()
test_oh = enc.transform(X_tes[cat_features]).toarray()

train_num = scaler.transform(X_res[num_features])
test_num = scaler.transform(X_tes[num_features])

print(train_oh.shape)
print(test_oh.shape)


(1260, 58)
(226, 58)


In [ ]:
X_trn = np.concatenate([train_num,train_oh,X_res['workday'].values.reshape(-1,1)], axis = 1)
X_tes = np.concatenate([test_num,test_oh,X_tes['workday'].values.reshape(-1,1)], axis = 1)

print(X_trn.shape)
print(X_tes.shape)

(1260, 67)
(226, 67)


In [ ]:
class TrnDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [ ]:
class TesDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [ ]:
train_data = TrnDataset(X_trn, y_res.values)
test_data = TesDataset(X_tes, y_tes.values)

train_loader = DataLoader(train_data,batch_size=100,shuffle=True)
test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [ ]:
arr, label = next(iter(train_loader))
print(arr.shape, label.shape)
arr2, label2 = next(iter(test_loader))
print(arr2.shape, label2.shape)

torch.Size([100, 67]) torch.Size([100])
torch.Size([100, 67]) torch.Size([100])


In [ ]:
latent_size = 4

In [ ]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(67,64),
                      nn.ReLU(),
                      nn.Linear(64,32),
                      nn.ReLU(),
                      nn.Linear(32,32),
                      nn.ReLU(),
                      nn.Linear(32,16),
                      nn.ReLU(),
                      nn.Linear(16,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())

  def forward(self,x):
    return self.encoder(x)


class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size, 16),
                      nn.ReLU(),
                      nn.Linear(16,16),
                      nn.ReLU(),
                      nn.Linear(16,32),
                      nn.ReLU(),
                      nn.Linear(32,32),
                      nn.ReLU(),
                      nn.Linear(32,64),
                      nn.ReLU(),
                      nn.Linear(64,67),
                      nn.Sigmoid())

  def forward(self,x):
    return self.decoder(x)

In [ ]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr = 0.0005)

In [ ]:
for ep in range(200):

  running_loss = 0.0

  for images, label in train_loader:
    inputs = images.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    outputs = decoder(latent_var)
    loss = criterion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  cost = running_loss/len(train_loader)
  print('[%d]loss:%.3f'%(ep,cost))
  torch.save(encoder.state_dict(), './models/en_pth')
  torch.save(decoder.state_dict(), './models/de_pth')

[0]loss:0.231
[1]loss:0.230
[2]loss:0.229
[3]loss:0.228
[4]loss:0.227
[5]loss:0.226
[6]loss:0.225
[7]loss:0.223
[8]loss:0.222
[9]loss:0.220
[10]loss:0.219
[11]loss:0.217
[12]loss:0.214
[13]loss:0.212
[14]loss:0.209
[15]loss:0.206
[16]loss:0.202
[17]loss:0.197
[18]loss:0.192
[19]loss:0.187
[20]loss:0.179
[21]loss:0.170
[22]loss:0.159
[23]loss:0.143
[24]loss:0.123
[25]loss:0.102
[26]loss:0.086
[27]loss:0.079
[28]loss:0.076
[29]loss:0.075
[30]loss:0.073
[31]loss:0.072
[32]loss:0.072
[33]loss:0.071
[34]loss:0.071
[35]loss:0.071
[36]loss:0.070
[37]loss:0.070
[38]loss:0.070
[39]loss:0.070
[40]loss:0.070
[41]loss:0.070
[42]loss:0.070
[43]loss:0.070
[44]loss:0.070
[45]loss:0.070
[46]loss:0.070
[47]loss:0.070
[48]loss:0.070
[49]loss:0.070
[50]loss:0.070
[51]loss:0.070
[52]loss:0.070
[53]loss:0.070
[54]loss:0.070
[55]loss:0.070
[56]loss:0.070
[57]loss:0.070
[58]loss:0.070
[59]loss:0.070
[60]loss:0.070
[61]loss:0.070
[62]loss:0.070
[63]loss:0.070
[64]loss:0.070
[65]loss:0.070
[66]loss:0.070
[67]l

In [ ]:
#forward training data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_trn = []
actual_trn = []

with torch.no_grad():
  for images, labels in train_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_trn+=latent_var.cpu().tolist()
    actual_trn+=labels.cpu().tolist()

In [ ]:
#forward test data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_tes = []
actual_tes = []

with torch.no_grad():
  for images, labels in test_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_tes+=latent_var.cpu().tolist()
    actual_tes+=labels.cpu().tolist()

In [ ]:
latent_features_trn = np.array(latent_features_trn)
latent_features_tes = np.array(latent_features_tes)
actual_tes = np.array(actual_tes)
actual_trn = np.array(actual_trn)

In [ ]:
latent_features_trn=pd.DataFrame(latent_features_trn)
latent_features_tes=pd.DataFrame(latent_features_tes)
actual_tes = pd.DataFrame(actual_tes)
actual_trn = pd.DataFrame(actual_trn)

In [ ]:
latent_features_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 3/latent_features_trn.csv')
latent_features_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 3/latent_features_tes.csv')
actual_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 3/actual_tes.csv')
actual_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 3/actual_trn.csv')

In [ ]:
actual_trn = actual_trn.values.reshape(-1,)
actual_tes = actual_tes.values.reshape(-1,)

In [ ]:
clf = RandomForestClassifier(n_estimators = 300, random_state = 0)
clf.fit(latent_features_trn, actual_trn)
y_trn_pred = clf.predict(latent_features_trn)

print(classification_report(actual_trn,y_trn_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       630
           1       0.97      0.97      0.97       630

    accuracy                           0.97      1260
   macro avg       0.97      0.97      0.97      1260
weighted avg       0.97      0.97      0.97      1260



In [ ]:
y_pred = clf.predict(latent_features_tes)
print(classification_report(actual_tes,y_pred))

              precision    recall  f1-score   support

           0       0.72      0.53      0.61       158
           1       0.32      0.51      0.40        68

    accuracy                           0.53       226
   macro avg       0.52      0.52      0.50       226
weighted avg       0.60      0.53      0.55       226

